# Goals / Plan

- Using regret for the game works, but is slow
    - choosing 1 random contract per round only, 1 random action per player
    - regret is inaccurance b/c it is based on a random sample from opponent's mixed strategies; imprecise

- How can we apply regret minimization WITHOUT random sampling (use precise probabilities)?

# Symmetric Regret and Convolutions

- example scenario:
    - contract is uniform [700, 1300]
    - contract is N(1000, 150) - bonus
    - 10 players for both

- initializations:
    - realistically, assume nobody is bidding over 200
    - initialize regret of options 1-200 np.zeros(200) --> even probabilities

- calculating regret:
    - assume we have a 200-length probability vector p
    - we want to find the regret of choosing each option based on MIXED STRATEGIES of opponents + CONTRACT PDF
        - this is the key, not using random samples, but exact probabilities

regret of choosing a value x:
- convolution of the other 9 players pdfs over values < x:
    - length x, convolve 9 times --> 9x log(9x)
    - this is the distribution of the sum of other players' lower bids
- how to integrate contract pdf
    - integral of [(contract == c) * (convolution cdf < c - x)]
    - don't calculate integral, do it discrete
    - for uniform distribution, calculate each contract value, then average (600 scenarios)

time complexity of calculating regret for value x:
- convolution takes 9x log (9x) time
- turn pdf into cdf (prefix sum linear time, 9x)
- for each contract scenario, find sum of probabilities (600 time)
- overall: 9x log(9x) * 600

time complexity over all values:
- Sum x=1 to x=200 of (9x log(9x) * 600) = 10^8 about
- Expressed differently: about max x^2 * players * contract length
- Assuming max x is about max(contract) / players * 2
- O(max(contract)^2 * contract length)

change to contract is [350, 650], 10 players (still feels like enough granularity)
- now instead of 10^8, about 10^7 per iteration
- iterations converge much quicker, about 100 iterations maybe?